In [2]:
import pandas as pd
import numpy as np
from tmdbv3api import TMDb
from tmdbv3api import Movie
from dotenv import load_dotenv
import os
import requests
from datetime import datetime

In [3]:
load_dotenv()

True

In [4]:
tmdb = TMDb()
tmdb.api_key = os.getenv('API_KEY')
tmdb.language = 'en'
tmdb.debug = True

In [5]:
movie = Movie()

### Extracting the names of all the movies from 2021

In [6]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_American_films_of_2021")

In [9]:
names = []

In [10]:
for i in range(2, 6):
    for i in data[i]['Title']:
        names.append(i)

In [27]:
movie_id = []
movie_not_found = []

In [28]:
date_format = "%Y-%m-%d"
start_date = datetime.strptime("2021-01-01", date_format)
end_date = datetime.strptime("2021-12-31", date_format)
for name in names:
    search = movie.search(name)
    for res in search:
        if(type(res) == str):
            print(res)
            break
        if(res['release_date']):
            if (datetime.strptime(res['release_date'], date_format) >= start_date and datetime.strptime(res['release_date'], date_format) <= end_date):
                movie_id.append((name, res['id']))
                break      
        else :
            print("Does not exist")
    else :
        print("Removed " + name + " from the names list")
        movie_not_found.append(name)
        # names.remove('name')

Removed Shadow in the Cloud from the names list
Does not exist
Does not exist
Removed Hacksaw from the names list
Does not exist
Removed American Skin from the names list
Removed Don't Tell a Soul from the names list
Does not exist
Does not exist
Removed Our Friend from the names list
Removed The SpongeBob Movie: Sponge on the Run from the names list
Removed Boss Level from the names list
Does not exist
Removed City of Lies from the names list
Removed Concrete Cowboy from the names list
Removed The Paper Tigers from the names list
Removed Above Suspicion from the names list
Removed Reboot Camp from the names list
Removed Benny Loves You from the names list
Does not exist
Removed Georgetown from the names list
Does not exist
Removed Changing the Game from the names list
Does not exist
Removed Zola from the names list
Removed Joe Bell from the names list
Removed She Ball from the names list
Removed Really Love from the names list
page
Does not exist
Removed Friend of the World from the n

In [31]:
details = []

In [32]:
def get_genres(obj):
    genres = []

    # print(type(obj))
    for i in obj:
        # print(i['name'])
        genres.append(i['name'])
    return genres

In [33]:
for i in movie_id:

    url = f'https://api.themoviedb.org/3/movie/{i[1]}?api_key={os.getenv("API_KEY")}'
    response = requests.get(url)
    data_json = response.json()
    movie_details = {}
    movie_details['name'] = data_json['title']
    movie_details['id'] = i[1]
    movie_details['overview'] = data_json['overview'] 
    movie_details['genres'] = get_genres(data_json['genres'])
    details.append(movie_details)


In [35]:
df = pd.DataFrame(details)

In [36]:
df.head()

,name,id,overview,genres
0,The White Tiger,628534,An ambitious Indian driver uses his wit and cu...,[Drama]
1,Locked Down,741228,"During a COVID-19 lockdown, sparring couple Li...","[Comedy, Crime, Drama]"
2,The Dig,532865,"As WWII looms, a wealthy widow hires an amateu...","[Drama, History]"
3,Outside the Wire,775996,"In the near future, a drone pilot is sent into...","[Thriller, Action, Science Fiction]"
4,The Marksman,634528,Jim Hanson’s quiet life is suddenly disturbed ...,[Action]


In [37]:
def get_actors(id):
    url  =  f'https://api.themoviedb.org/3/movie/{id}/credits?api_key={os.getenv("API_KEY")}'
    resonse = requests.get(url)
    data_json = resonse.json()
    obj = data_json['cast']
    actors = []
    length = 5 if len(obj) > 5 else len(obj)
    for i in range(length):
        actors.append(obj[i]['name'])
    return actors

In [38]:
def get_directors(id):
    url  =  f'https://api.themoviedb.org/3/movie/{id}/credits?api_key={os.getenv("API_KEY")}'
    resonse = requests.get(url)
    data_json = resonse.json()
    obj = data_json['crew']
    directors = []
    for i in obj:
        if(i['job'] == 'Director'):
            directors.append(i['name'])
            break
    return directors

In [39]:
df['actors'] = df['id'].apply(get_actors)

In [40]:
df.head()

,name,id,overview,genres,actors
0,The White Tiger,628534,An ambitious Indian driver uses his wit and cu...,[Drama],"[Adarsh Gourav, Rajkummar Rao, Priyanka Chopra..."
1,Locked Down,741228,"During a COVID-19 lockdown, sparring couple Li...","[Comedy, Crime, Drama]","[Anne Hathaway, Chiwetel Ejiofor, Stephen Merc..."
2,The Dig,532865,"As WWII looms, a wealthy widow hires an amateu...","[Drama, History]","[Carey Mulligan, Ralph Fiennes, Lily James, Jo..."
3,Outside the Wire,775996,"In the near future, a drone pilot is sent into...","[Thriller, Action, Science Fiction]","[Anthony Mackie, Damson Idris, Enzo Cilenti, E..."
4,The Marksman,634528,Jim Hanson’s quiet life is suddenly disturbed ...,[Action],"[Liam Neeson, Jacob Perez, Harry Maldonado, Te..."


In [41]:
df['director'] = df['id'].apply(get_directors)

In [42]:
df.head()

,name,id,overview,genres,actors,director
0,The White Tiger,628534,An ambitious Indian driver uses his wit and cu...,[Drama],"[Adarsh Gourav, Rajkummar Rao, Priyanka Chopra...",[Ramin Bahrani]
1,Locked Down,741228,"During a COVID-19 lockdown, sparring couple Li...","[Comedy, Crime, Drama]","[Anne Hathaway, Chiwetel Ejiofor, Stephen Merc...",[Doug Liman]
2,The Dig,532865,"As WWII looms, a wealthy widow hires an amateu...","[Drama, History]","[Carey Mulligan, Ralph Fiennes, Lily James, Jo...",[Simon Stone]
3,Outside the Wire,775996,"In the near future, a drone pilot is sent into...","[Thriller, Action, Science Fiction]","[Anthony Mackie, Damson Idris, Enzo Cilenti, E...",[Mikael Håfström]
4,The Marksman,634528,Jim Hanson’s quiet life is suddenly disturbed ...,[Action],"[Liam Neeson, Jacob Perez, Harry Maldonado, Te...",[Robert Lorenz]


In [43]:
def remove_space(obj):
    l = []
    for i in obj:
        l.append(i.replace(" ", ""))
    return l

In [47]:
df['tags'] = df['tags'].apply(lambda x : " ".join(x))

In [48]:
df.head()

,name,id,overview,genres,actors,director,tags
0,The White Tiger,628534,"[An, ambitious, Indian, driver, uses, his, wit...",[Drama],"[AdarshGourav, RajkummarRao, PriyankaChopraJon...",[RaminBahrani],An ambitious Indian driver uses his wit and cu...
1,Locked Down,741228,"[During, a, COVID-19, lockdown,, sparring, cou...","[Comedy, Crime, Drama]","[AnneHathaway, ChiwetelEjiofor, StephenMerchan...",[DougLiman],"During a COVID-19 lockdown, sparring couple Li..."
2,The Dig,532865,"[As, WWII, looms,, a, wealthy, widow, hires, a...","[Drama, History]","[CareyMulligan, RalphFiennes, LilyJames, Johnn...",[SimonStone],"As WWII looms, a wealthy widow hires an amateu..."
3,Outside the Wire,775996,"[In, the, near, future,, a, drone, pilot, is, ...","[Thriller, Action, Science Fiction]","[AnthonyMackie, DamsonIdris, EnzoCilenti, Emil...",[MikaelHåfström],"In the near future, a drone pilot is sent into..."
4,The Marksman,634528,"[Jim, Hanson’s, quiet, life, is, suddenly, dis...",[Action],"[LiamNeeson, JacobPerez, HarryMaldonado, Teres...",[RobertLorenz],Jim Hanson’s quiet life is suddenly disturbed ...


In [49]:
new_df = df[['id', 'name', 'tags']]

In [50]:
new_df

,id,name,tags
0,628534,The White Tiger,An ambitious Indian driver uses his wit and cu...
1,741228,Locked Down,"During a COVID-19 lockdown, sparring couple Li..."
2,532865,The Dig,"As WWII looms, a wealthy widow hires an amateu..."
3,775996,Outside the Wire,"In the near future, a drone pilot is sent into..."
4,634528,The Marksman,Jim Hanson’s quiet life is suddenly disturbed ...
...,...,...,...
335,476669,The King's Man,As a collection of history's worst tyrants and...
336,591538,The Tragedy of Macbeth,"Macbeth, the Thane of Glamis, receives a proph..."
337,501841,A Journal for Jordan,Based on the true story of First Sergeant Char...
338,673309,American Underdog,"The true story of Kurt Warner, who went from a..."


In [51]:
df = new_df.rename(columns = {'id' : 'movie_id', 'name' : 'title'})

In [52]:
df.head()

,movie_id,title,tags
0,628534,The White Tiger,An ambitious Indian driver uses his wit and cu...
1,741228,Locked Down,"During a COVID-19 lockdown, sparring couple Li..."
2,532865,The Dig,"As WWII looms, a wealthy widow hires an amateu..."
3,775996,Outside the Wire,"In the near future, a drone pilot is sent into..."
4,634528,The Marksman,Jim Hanson’s quiet life is suddenly disturbed ...


In [54]:
df.to_csv('data_2021.csv', index = False)